In [ ]:
#data management and custom functions
from pathlib import Path
from util import pil_imread
import tifffile as tf
from chromatic_aberration_correction import *
#plotting packages
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

# Max project each channel if desired before running chromatic aberration correction.

In [ ]:
from pathlib import Path
#read in corrected images
num_pos = 3
hyb_num = 37
for pos in range(num_pos):
    for hyb in range(hyb_num):
        src = f"/groups/CaiLab/personal/Lex/raw/Ratio_metric_Lantern/05312022_automation/notebook_pyfiles/dapi_aligned/HybCycle_{hyb}/MMStack_Pos{pos}.ome.tif"
        img = pil_imread(src,swapaxes=True)
        img = np.max(img,axis=0)
        output_dir = Path(src).parent.parent.parent
        output_dir = output_dir / "max_projected"
        img_name = Path(src).name
        hyb = Path(src).parent.name
        output_path = output_dir / hyb / img_name
        output_path.parent.mkdir(parents=True, exist_ok=True)
        tf.imwrite(str(output_path), img)

# Path to ref image for testing

In [ ]:
tiff_src = "/groups/CaiLab/personal/Lex/raw/Ratio_metric_repeat/notebook_pyfiles/max_projected/EEF2_ratiometric_Rlvl6_Llvl2_premixed_730,640,561,488,405_2000,200,300,400,400_Pos2.ome.tif"

# Get offsets and corrected image

In [ ]:
import time
start = time.time()
transformed_image, error, tform = chromatic_corr_offsets(tiff_src, threshold_abs=500,
                                                         max_dist=2, ransac_threshold = 1, swapaxes=True)
print(f"This task took {(time.time()-start)/60} min")

In [ ]:
#alignment error
error

In [ ]:
#look at transformation matrix
tform

# Compare corrected and original image

In [ ]:
def plot_2d_image(img_2d, zmax=1000, animation = True):
    
    if animation == True:   
    #For Plotting 2d image
        #-------------------------------------------
        fig = px.imshow(
            img_2d,
            width=700,
            height=700,
            binary_string=True,
            binary_compression_level=4,
            binary_backend='pil',
            zmax = zmax,
            animation_frame=0
        )
        #-------------------------------------------

        fig.show()
    else:
        #For Plotting 2d image
        #-------------------------------------------
        fig = px.imshow(
            img_2d,
            width=700,
            height=700,
            binary_string=True,
            binary_compression_level=4,
            binary_backend='pil',
            zmax = zmax,
        )
        #-------------------------------------------

        fig.show()

In [ ]:
#read in images
original = pil_imread(tiff_src, swapaxes=True)
original_max= np.max(original, axis=0)
transformed_max = np.max(transformed_image,axis=0)

In [ ]:
#original
plot_2d_image(original_max, zmax=4000, animation = True)

In [ ]:
#corrected
plot_2d_image(transformed_max, zmax=4000, animation = True)

# Check average error

In [ ]:
poss = np.arange(0,60,1)
error_list = []
for pos in poss:
    src = f"/groups/CaiLab/personal/Lex/raw/020822_erna_dash_tf/chromatic_aberration/MMStack_Pos{pos}_error.txt"
    error = pd.read_csv(src, sep = " ", header=None)
    error_list.append(error)

In [ ]:
comb_error = pd.concat(error_list)
comb_error.columns = ["channel", "percent improvement", "fwhm"]

In [ ]:
comb_error.groupby("channel").mean()

In [ ]:
comb_error.groupby("channel").std()

# Apply tform (use this to trouble shoot if ref tform is off)

In [ ]:
# from pathlib import Path
# from webfish_tools.util import find_matching_files

In [ ]:
# JOB_ID = 0
# directory = Path("/groups/CaiLab/personal/Lex/raw/112221_20kdash_3t3/notebook_pyfiles/dapi_aligned/")
# position_name = f'MMStack_Pos{JOB_ID}.ome.tif'

In [ ]:
# #hybcycle images
# files, _, _ = find_matching_files(directory, 'HybCycle_{hyb}' + f'/{position_name}')
# files = [str(f) for f in files]

In [ ]:
# files

In [ ]:
# #reference image
# ref_directory = "/groups/CaiLab/personal/Lex/raw/112221_20kdash_3t3/chromatic_aberration_lb/new_probes_5/new_probes_5_MMStack_Pos0.ome.tif"

# #get offsets
# _, _, tform = chromatic_corr_offsets(ref_directory,region_size=7, min_distance=10, 
#                           threshold_abs=500, num_peaks=500, max_dist=5,
#                           include_dapi=False, swapaxes=True)

# #apply offsets
# apply_chromatic_corr(ref_directory, tform, cores = 12, include_dapi=False, swapaxes=True, write = True)

In [ ]:
# #read in corrected
# hyb_corr = pil_imread("/groups/CaiLab/personal/Lex/raw/112221_20kdash_3t3/notebook_pyfiles/aberration_corrected/HybCycle_2/MMStack_Pos0.ome.tif")
# #corrected
# plot_2d_image(hyb_corr[0], zmax=4000, animation = True)

In [ ]:
# #read in corrected
# hyb_corr = pil_imread("/groups/CaiLab/personal/Lex/raw/112221_20kdash_3t3/aberration_corrected/new_probes_5/new_probes_5_MMStack_Pos0.ome.tif")
# hyb_corr = np.max(hyb_corr, axis=0)
# #corrected
# plot_2d_image(hyb_corr, zmax=4000, animation = True)